# Stylometry - model training

### Used classifiers
 - Decision Tree
 - kNN (NYI)
 - SVM (NYI)
 
 
 - ? Logistic Regression (NYI)

## Imports

In [41]:
import matplotlib
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats.api as sms
from sklearn import preprocessing
from dateutil import parser
import statistics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, mutual_info_regression, chi2, f_regression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.impute import KNNImputer
from sklearn import tree, svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import metrics
from skmultilearn.model_selection import IterativeStratification
from sklearn.linear_model import LogisticRegression
import gc
import warnings
warnings.filterwarnings('ignore')

## Helper functions

In [42]:
def scale(df_to_scale, ignored_cols):
    cols_to_scale = df_to_scale.drop(ignored_cols , axis=1).columns
    min_max_scaler = MinMaxScaler()
    df_to_scale[cols_to_scale] = min_max_scaler.fit_transform(df_to_scale[cols_to_scale])
    df.info()
    return df_to_scale

In [43]:
def evaluate(real, predicted):
    print("Evaluation: ")
    countGood = 0
    total = len(real)
    falsePos = 0
    falseNeg = 0
    truePos = 0
    trueNeg = 0
    if total != len(predicted):
        print("Non-matching sample lengths")
        return None
    for i in range(0, total):
        countGood += 1 if real[i] == predicted[i] else 0

    # accuracy = str(countGood / total)
    # precision = str(truePos / (truePos + falsePos))
    # recall = str(truePos / (truePos + falseNeg))
    
    print("accuracy " + str(countGood / total))
    return countGood / total

## Data import
\+calculate positional error occurence per token

In [82]:

df = pd.read_csv('sample_100articles_3grams_swaps.csv', sep=',')
df['Toperror_pertokens'] = df['Toperror'] / df['Tokens']
df['Lefterror_pertokens'] = df['Lefterror'] / df['Tokens']
df['Righterror_pertokens'] = df['Righterror'] / df['Tokens']
df['Downerror_pertokens'] = df['Downerror'] / df['Tokens']
df.info()
df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Columns: 2304 entries, Author to Downerror_pertokens
dtypes: float64(2267), int64(36), object(1)
memory usage: 31.6+ MB


,Author,UppercaseLetter,UppercaseLetter_pertokens,LowercaseLetter,LowercaseLetter_pertokens,TitlecaseLetter,TitlecaseLetter_pertokens,ModifierLetter,ModifierLetter_pertokens,OtherLetter,...,swap_ _m,swap_ _CO,swap_ _.,swap_ _/,swap_ _,Tokens,Toperror_pertokens,Lefterror_pertokens,Righterror_pertokens,Downerror_pertokens
0,1021,1,0.071,59,4.214,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,14,0.0,0.000000,0.0,0.0
1,1021,2,0.182,45,4.091,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,11,0.0,0.000000,0.0,0.0
2,1021,11,0.229,99,2.063,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,48,0.0,0.041667,0.0,0.0


In [83]:
df = df.replace("∞", "0")

In [84]:
# cols_to_scale = df.drop(["Author", "Tokens"] , axis=1).columns
# df_all = df
# min_max_scaler = MinMaxScaler()
# df_all[cols_to_scale] = min_max_scaler.fit_transform(df_all[cols_to_scale])
df_all = scale(df, ["Author", "Tokens"])
df_all

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Columns: 2304 entries, Author to Downerror_pertokens
dtypes: float64(2302), int64(2)
memory usage: 31.6 MB


,Author,UppercaseLetter,UppercaseLetter_pertokens,LowercaseLetter,LowercaseLetter_pertokens,TitlecaseLetter,TitlecaseLetter_pertokens,ModifierLetter,ModifierLetter_pertokens,OtherLetter,...,swap_ _m,swap_ _CO,swap_ _.,swap_ _/,swap_ _,Tokens,Toperror_pertokens,Lefterror_pertokens,Righterror_pertokens,Downerror_pertokens
0,1021,0.001239,0.017212,0.002489,0.491658,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,14,0.0,0.000000,0.000000,0.000000
1,1021,0.002478,0.044121,0.001898,0.477307,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11,0.0,0.000000,0.000000,0.000000
2,1021,0.013631,0.055515,0.004176,0.240695,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,48,0.0,0.166667,0.000000,0.000000
3,1021,0.004957,0.042182,0.003712,0.446389,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,23,0.0,0.000000,0.086957,0.000000
4,1021,0.018587,0.038303,0.015650,0.455606,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,95,0.0,0.084211,0.000000,0.075439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,1317,0.049566,0.027636,0.049566,0.389453,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,352,0.0,0.011364,0.000000,0.000000
1796,1317,0.013631,0.031030,0.012866,0.413837,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,86,0.0,0.000000,0.000000,0.000000
1797,1317,0.004957,0.026909,0.006074,0.466690,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,36,0.0,0.111111,0.000000,0.000000
1798,1317,0.016109,0.017697,0.024804,0.385369,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,178,0.0,0.000000,0.005618,0.000000


In [85]:
len(df_all['Author'].unique())

18

## Dataframes
df_all - contains all feature columns and author id

df_positional - contains positional errors, token count, misspell ratio and author id

non_empty_cols - all columns with non-zero values

In [86]:
all_cols = df_all.columns
to_add_cols = [  'Author',
                 'CurrencySymbol_pertokens',
                 'ConnectorPunctuation_pertokens',
                 'Downerror_pertokens',
                 'DecimalDigitNumber_pertokens',
                 'FinalQuotePunctuation_pertokens',
                 'InitialQuotePunctuation_pertokens',
                 'LowercaseLetter_pertokens',
                 'Lefterror_pertokens',
                 'MathSymbol_pertokens',
                 'Misspellratio',
                 'OtherLetter_pertokens',
                 'OtherNotAssigned_pertokens',
                 'Righterror_pertokens',
                 'SpaceSeparator_pertokens',
                 'Toperror_pertokens',
                 'UppercaseLetter_pertokens']


# non-empty PoS tag columns
prefixes = ['pos_',
            'swap_']
for col in all_cols:
    if (not((df_all[col] == 0).all()) and any(col.startswith(x) for x in prefixes)):
        to_add_cols.append(col)
df_positional = df_all[to_add_cols]

### Filtering
removing NaN and inf values

In [87]:
df_positional.dropna()
indices_to_keep = ~df_positional.isin([np.nan, np.inf, -np.inf]).any(1)
df_positional = df_positional[indices_to_keep].astype(np.float64)
df_positional.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1797 entries, 0 to 1799
Columns: 1954 entries, Author to swap_ _ 
dtypes: float64(1954)
memory usage: 26.8 MB


## OPTIONAL 
+ used to reduce the number of authors

In [91]:
splits = 1

print(len(df_positional["Author"].unique()))
if splits != 1:
    all_authors = df_all["Author"].unique()
    split_authors = all_authors[0:len(all_authors)//1]
    df_positional = df_positional[(df_positional["Author"] >= split_authors[0]) 
                                    & (df_positional["Author"] <= split_authors[-1])]
    print(len(df_positional["Author"].unique()))

18


# Split to train/test
!TODO, only taking a random sample and testing it on itself

In [88]:
data_count = len(df_positional)
percentage_to_train = 0.70
df_train = df_positional.sample(int(data_count*(percentage_to_train)))
df_test = pd.concat([df_train, df_positional]).drop_duplicates(keep=False)
# df_test = df_positional[int(data_count*2/3):]
df_train.info()
df_test.info()
print(data_count)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1257 entries, 908 to 28
Columns: 1954 entries, Author to swap_ _ 
dtypes: float64(1954)
memory usage: 18.7 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 0 to 1796
Columns: 1954 entries, Author to swap_ _ 
dtypes: float64(1954)
memory usage: 8.0 MB
1797


### Split training into features and labels

In [89]:
X = df_train.drop('Author', axis=1)
y = df_train["Author"]
len(X)

1257

# Model fitting
### Decision Tree Classifier
Create a decision tree classifier and get predictions on the same dataset

In [98]:
dt = tree.DecisionTreeClassifier(random_state=3)
dt = dt.fit(X, y)

In [99]:
real_authors = df_train["Author"].to_numpy()

### Logistic Regression

In [100]:
# model = LogisticRegression(multi_class='multinomial', random_state=0)
# model.fit(X, y)

### Random forest

In [101]:
forest=RandomForestClassifier(n_estimators=200)
forest.fit(X,y)

RandomForestClassifier(n_estimators=200)

# Evaluation
Create features and labels from testing dataset

In [102]:
X_test = df_test.drop('Author', axis=1)
Y_test = df_test['Author'].to_numpy()

## DT

In [103]:
# Decision Tree
predictions_dt = dt.predict(X_test)
dt_acc = evaluate(Y_test, predictions_dt)

Evaluation: 
accuracy 0.37265917602996257


### LogReg

In [104]:
# predictions_logreg = model.predict(X_test)
# logreg_acc = evaluate(Y_test, predictions_logreg)

### Random Forest

In [105]:
predictions_forest = forest.predict(X_test)
forest_acc = evaluate(Y_test, predictions_forest)

Evaluation: 
accuracy 0.49063670411985016


In [106]:
print("Decision tree")
print(dt_acc)
# print("Logreg")
# print(logreg_acc)
print("Forest")
print(forest_acc)

Decision tree
0.37265917602996257
Forest
0.49063670411985016


# Other -------- UNRELATED -------
under construction

## Automatic feature selection

In [22]:
# def select_features(df, k=5):
#     df_new = df.drop(["Author"], axis=1)
#     indicators_indexed = []
#     for x in df['Author']:
#         indicators_indexed.append(x)
# #     print(len(indicators_indexed))
#     # print(df_normal)
#     np.random.seed(1)
#     selector = SelectKBest(chi2, k=k)
# #     selector = SelectKBest(mutual_info_regression, k=k)
#     k_best_features = selector.fit_transform(df_new.values, indicators_indexed)
#     print(selector.get_support(1))
#     return (k_best_features, selector.get_support())

In [23]:
# feat, mask = select_features(df_train, 6)

# new_features = df_train.drop(["Author"], axis=1).columns[mask]
# new_features
# # print(ind)
# df_positional = df_all[new_features]
# df_positional["Author"] = df_all["Author"]
# df_positional.info()